In [1]:
from pathlib import Path
from anndata import read_h5ad
import sys
import scanpy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import nbinom, pearsonr
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model
import random
from scipy import stats
import time
file = Path("/scratch/imb/Xiao/STimage/stimage").resolve()
parent= file.parent
sys.path.append(str(parent))
sys.path.insert(0, '/scratch/imb/Xiao/STimage')
from tqdm import tqdm
import pickle
from stimage._utils import gene_plot, Read10X, ReadOldST, tiling, ensembl_to_id
from stimage._model import CNN_NB_multiple_genes, negative_binomial_layer, negative_binomial_loss, PrinterCallback
from stimage._data_generator import DataGenerator

start = time.time()


DATA_PATH = Path("/scratch/imb/Xiao/STimage_100run/dataset_breast_cancer_9visium")
OUT_PATH = Path("/scratch/imb/Xiao/STimage/development/stimage_Pfizer_10run/results")
OUT_PATH.mkdir(parents=True, exist_ok=True)

In [2]:
# job_id = int(sys.argv[1])
job_id = 0
random.seed(job_id)
np.random.seed(job_id)
tf.random.set_seed(job_id)

In [3]:
adata_all = read_h5ad(DATA_PATH / "all_adata.h5ad")

adata_all.obs["tile_path"] = adata_all.obs.tile_path.map(
    lambda x: x.replace("/clusterdata/uqxtan9/Xiao/breast_cancer_9visium",
                        "/scratch/imb/Xiao/STimage_100run/dataset_breast_cancer_9visium"))


df_gene_ls = pd.read_csv("/scratch/imb/Xiao/STimage/development/stimage_Pfizer_10run/Intersection_marker_genes.csv", sep=",")

gene_list_select = df_gene_ls["gene_name"].values.tolist()

gene_list = adata_all.var_names.intersection(gene_list_select)

In [4]:
DATA_PATH = Path("/clusterdata/uqxtan9/Xiao/dataset_Pfizer")

adata_all_ = read_h5ad(DATA_PATH / "all_adata.h5ad")

adata_all = adata_all_[adata_all_.obs["library_id"].isin(
    adata_all_.obs.library_id.cat.remove_categories("block1").unique())]
samples = adata_all.obs["library_id"].unique().tolist()

In [5]:
K.clear_session()

model = None
file_model = f'/clusterdata/uqxtan9/Xiao/STimage/development/stimage_100run/results_1/model_{job_id}.h5'
model = load_model(file_model, 
                   custom_objects={
                       'negative_binomial_loss': negative_binomial_loss,
                   }
                  )
print("loaded")

loaded


In [ ]:
for sample in samples:
    adata = adata_all[adata_all.obs.library_id==sample, adata_all.var_names[0]]
    test_gen = tf.data.Dataset.from_generator(
                lambda:DataGenerator(adata=adata, 
                              genes=adata.var_names), 
                output_types=(tf.float32, tuple([tf.float32]*len(adata.var_names))), 
                output_shapes=([299,299,3], tuple([1]*len(adata.var_names))))
    test_gen_ = test_gen.batch(1)
    pred_test = model.predict(test_gen_)
    # save out results
    with open(str(OUT_PATH / f'ensemble_results_{sample}_{job_id}.pkl'), 'wb') as file:
        pickle.dump(pred_test, file)